# Benchmark structural regression 

In [3]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [185]:
# Parse scores
tc_scores = "/home/luisasantus/Desktop/crg_cluster/projects/structural_regression/results/CSV/friendly_poitras/friendly_poitras.dynamic.tcScore.csv"
header = list(["family", "type", "batches", "full", "aligner", "score"])
scores_df = pd.read_csv(tc_scores, sep = ";", header = None)
scores_df.set_axis(header, axis=1, inplace=True)




scores_df.head(2)

AttributeError: 'DataFrame' object has no attribute 'name'

In [ ]:
scores_df.full.str.split(";", expand = True)[1].str.replace(")", "", regex = False).str.replace("_msa", "").str.split(":", expand = True)[0]


In [202]:
scores_df.full.str.split("_", expand = True)

,0,1,2,3,4
0,tcoffee,msa.100,famsa,msa.100000000,uniref50
1,tcoffee,msa.100,famsa,msa.100000000,uniref50


In [182]:
# Parse computational time
tracks = "/home/luisasantus/Desktop/crg_cluster/projects/structural_regression/metainfo/pipeline_info/execution_trace.txt"
tracks_df = pd.read_csv(tracks,sep = "\t")
tracks_df = tracks_df[list(["name", "duration"])]
tracks_df = tracks_df[tracks_df.name.str.contains("DYNAMIC_ALIGNER")]
tracks_df["aligner"] = tracks_df.name.str.split(";", expand = True)[1].str.replace(")", "", regex = False).str.replace("_msa", "").str.split(":", expand = True)[0]
tracks_df["tree"] = tracks_df.name.str.split(";", expand = True)[1].str.replace(")", "",  regex = False).str.replace("_msa", "").str.split(":", expand = True)[1]
tracks_df["family"] = tracks_df.name.str.split("on", expand = True)[1].str.split(";", expand = True)[0]
tracks_df.head()

,name,duration,aligner,tree,family
1,pipeline:DYNAMIC_ANALYSIS:DYNAMIC_ALIGNER (CON...,6s,tcoffee-100,famsa-100000000,seatoxin
3,pipeline:DYNAMIC_ANALYSIS:DYNAMIC_ALIGNER (CON...,20.7s,tcoffee-100,famsa-100000000,hip


In [203]:
df = pd.merge(tracks_df, tracks_df,  on = list(["family", "aligner", "tree"]))
df.head()

In [170]:
tracks_df

,name,duration,aligner,tree
1,pipeline:DYNAMIC_ANALYSIS:DYNAMIC_ALIGNER (CON...,6s,tcoffee-100,famsa-100000000
3,pipeline:DYNAMIC_ANALYSIS:DYNAMIC_ALIGNER (CON...,20.7s,tcoffee-100,famsa-100000000


In [204]:
scores_df

,family,type,batches,full,aligner,score
0,seatoxin,dynamic,100_100000,tcoffee_msa.100_famsa_msa.100000000_uniref50,FAMSA,26.5
1,hip,dynamic,100_100000,tcoffee_msa.100_famsa_msa.100000000_uniref50,FAMSA,50.0


In [206]:
x = "tcoffee_msa.100_famsa_msa.100000000_uniref50"